In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore")

test_data = pd.read_csv(r"C:\Users\2610237L\OneDrive - University of Glasgow\1 Individual Project\Germany_Data_28-02_03-03.csv")
sarimax_predictions = pd.read_csv(r"C:\Users\2610237L\OneDrive - University of Glasgow\1 Individual Project\SARIMAX_Prediction_Data_28-02-2023-05-00_03-03-2023-05-00.csv")

test_data['Datetime'] = pd.to_datetime(test_data['Datetime'])
sarimax_predictions['Datetime'] = pd.to_datetime(sarimax_predictions['Datetime'])

data = test_data.merge(sarimax_predictions, on='Datetime', suffixes=('_actual', '_predicted'))
data.rename(columns={'Price (EUR/MWHE)': 'market_price', 'Predicted Price': 'market_price_predicted'}, inplace=True)

battery_capacity = 55  
initial_budget = 100  
starting_soc = 50  
charge_efficiency = 0.95
discharge_efficiency = 0.95
soc_limits = [10, 90]  

decisions = []
soc = starting_soc
budget = initial_budget
charge_indices, discharge_indices, hold_indices = [], [], []
soc_values, profits = [soc], [0]

def calculate_charge_percentage(current_price, predicted_price, action_type):
    price_ratio = predicted_price / current_price
    if action_type == 1:  
        if price_ratio > 1.25:
            return 0.25  
        elif price_ratio > 1.1:  
            return 0.1  
        elif price_ratio > 1.05:
            return 0.05
        else:
            return 0.02 
    elif action_type == -1:  
        if price_ratio < 0.9:
            return 0.1 
        elif price_ratio < 0.95:
            return 0.05  
        else:
            return 0.02  

for t in range(len(data) - 1):
    current_price = data['market_price'].iloc[t]
    predicted_price = data['market_price_predicted'].iloc[t + 1]
    price_ratio = predicted_price / current_price
    action = 0  

    if predicted_price > current_price and soc < soc_limits[1]:  
        action = 1  
    elif predicted_price < current_price and soc > soc_limits[0]:  
        action = -1  
    elif 0.95 <= price_ratio <= 1.05:  
        action = 0  

    if action > 0:  
        charge_percentage = calculate_charge_percentage(current_price, predicted_price, action)
        charge_amount = min((soc_limits[1] - soc) / 100 * battery_capacity, charge_percentage * battery_capacity)  
        budget -= current_price * charge_amount / charge_efficiency
        soc += (charge_amount * charge_efficiency / battery_capacity) * 100
        charge_indices.append(t)
    elif action < 0:  
        discharge_percentage = calculate_charge_percentage(current_price, predicted_price, action)
        discharge_amount = min((soc - soc_limits[0]) / 100 * battery_capacity, discharge_percentage * battery_capacity)
        budget += current_price * discharge_amount / discharge_efficiency
        soc -= (discharge_amount * discharge_efficiency / battery_capacity) * 100
        discharge_indices.append(t)
    else:
        hold_indices.append(t)
    
    soc = max(soc_limits[0], min(soc_limits[1], soc))
    soc_values.append(soc)
    profits.append(budget - initial_budget)

plt.figure(figsize=(12, 6))
plt.plot(range(len(data)), data["market_price"], label="Market Price")
plt.scatter(charge_indices, data["market_price"].iloc[charge_indices], color="green", label="Charge")
plt.scatter(discharge_indices, data["market_price"].iloc[discharge_indices], color="red", label="Discharge")
plt.plot(range(len(data)), data["market_price_predicted"], label="Predicted Price", linestyle=":")
plt.xlabel("Index")
plt.ylabel("Price (€/kWh)")
plt.legend(loc=1)
plt.grid()
plt.show()

plt.figure(figsize=(12, 6))
plt.plot(range(len(profits)), profits, label="Cumulative Profit (€)", color="orange")
plt.axhline(0, color="red", linestyle="--", label="Break-even")
plt.xlabel("Index")
plt.ylabel("Profit (€)")
plt.legend(loc=1)
plt.grid()
plt.show()

print(f"Final budget: €{budget:.2f}")
print(f"Net profit: €{budget - initial_budget:.2f}")